In [1]:
# 社區自動對帳系統 (Auto Reconcile Pro)
# 主程式：auto_reconcile_pro.py
# 功能：PDF OCR 辨識 + Excel 比對 + 自動寄信 + 每月歸檔 + GUI介面

import os, re, cv2, pytesseract, smtplib
import pandas as pd
from pdf2image import convert_from_path
from PIL import Image
from tkinter import Tk, Label, Button, filedialog, messagebox, Checkbutton, IntVar
from email.message import EmailMessage
from datetime import datetime

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
    return Image.fromarray(thresh)

def extract_info_from_text(text):
    accounts = re.findall(r'帳號[:： ]?(\d{10,14})', text)
    amounts = re.findall(r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', text)
    dates = re.findall(r'\d{4}/\d{1,2}/\d{1,2}', text)
    return [{"帳號": a, "金額": b, "日期": c} for a, b, c in zip(accounts, amounts, dates)]

def process_pdfs(pdf_folder):
    all_data = []
    pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]
    if not pdf_files:
        messagebox.showwarning("警告", "資料夾內沒有找到 PDF 檔案！")
        return pd.DataFrame()
    for pdf_file in pdf_files:
        pages = convert_from_path(os.path.join(pdf_folder, pdf_file), dpi=300)
        for i, page in enumerate(pages):
            img_path = f"temp_{i}.jpg"
            page.save(img_path, "JPEG")
            img = preprocess_image(img_path)
            text = pytesseract.image_to_string(img, lang="chi_tra+eng")
            all_data.extend(extract_info_from_text(text))
            os.remove(img_path)
    return pd.DataFrame(all_data)

def export_excel(df_resident, df_bank, base_folder):
    result = pd.merge(df_resident, df_bank, on="帳號", how="left", indicator=True)
    result["繳費狀態"] = result["_merge"].map({"both": "✅ 已繳", "left_only": "❌ 未繳"})
    result.drop(columns=["_merge"], inplace=True)

    now = datetime.now()
    month_folder = os.path.join(base_folder, f"{now.year}-{now.month:02d}_對帳報告")
    os.makedirs(month_folder, exist_ok=True)

    output_file = os.path.join(month_folder, f"社區自動對帳報告_{now.strftime('%Y%m%d')}.xlsx")
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        result[result["繳費狀態"] == "✅ 已繳"].to_excel(writer, sheet_name="已繳住戶", index=False)
        result[result["繳費狀態"] == "❌ 未繳"].to_excel(writer, sheet_name="未繳住戶", index=False)
        result["月份"] = result["日期"].dropna().apply(lambda x: str(x)[:7] if isinstance(x, str) else "")
        for month, group in result.groupby("月份"):
            if month:
                group.to_excel(writer, sheet_name=f"{month}明細", index=False)
    return output_file

def send_email_report(sender, password, receiver, file_path):
    msg = EmailMessage()
    msg["Subject"] = f"社區自動對帳報告 - {datetime.now().strftime('%Y/%m/%d')}"
    msg["From"] = sender
    msg["To"] = receiver
    msg.set_content("附件為本期對帳報告，請查收。")
    with open(file_path, "rb") as f:
        msg.add_attachment(f.read(),
            maintype="application",
            subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            filename=os.path.basename(file_path))
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(sender, password)
        smtp.send_message(msg)

class ReconcileApp:
    def __init__(self, root):
        self.root = root
        self.root.title("社區自動對帳系統")
        self.root.geometry("520x380")
        self.pdf_folder = ""
        self.excel_file = ""
        self.send_email = IntVar()

        Label(root, text="社區自動對帳系統", font=("微軟正黑體", 16, "bold")).pack(pady=10)
        Button(root, text="📂 選擇存摺PDF資料夾", command=self.select_pdf_folder, width=30).pack(pady=10)
        Button(root, text="📘 選擇住戶名單Excel檔", command=self.select_excel_file, width=30).pack(pady=10)
        Checkbutton(root, text="📧 自動寄出對帳報告 (Gmail)", variable=self.send_email).pack(pady=5)
        Button(root, text="▶️ 開始對帳", command=self.start_process, width=20, bg="#0d6efd", fg="white").pack(pady=20)

    def select_pdf_folder(self):
        folder = filedialog.askdirectory(title="選擇PDF資料夾")
        if folder:
            self.pdf_folder = folder
            messagebox.showinfo("已選取", f"PDF 資料夾：\n{folder}")

    def select_excel_file(self):
        file = filedialog.askopenfilename(title="選擇Excel檔", filetypes=[("Excel files", "*.xlsx")])
        if file:
            self.excel_file = file
            messagebox.showinfo("已選取", f"Excel 名單：\n{file}")

    def start_process(self):
        if not self.pdf_folder or not self.excel_file:
            messagebox.showwarning("警告", "請先選擇 PDF 資料夾與 Excel 名單！")
            return
        df_bank = process_pdfs(self.pdf_folder)
        if df_bank.empty:
            return
        df_resident = pd.read_excel(self.excel_file)
        df_resident["帳號"] = df_resident["帳號"].astype(str).str.replace("-", "").str.strip()
        df_bank["帳號"] = df_bank["帳號"].astype(str).str.replace("-", "").str.strip()
        report_path = export_excel(df_resident, df_bank, base_folder=os.getcwd())
        messagebox.showinfo("完成", f"報告已輸出：\n{report_path}")
        if self.send_email.get() == 1:
            self.send_email_dialog(report_path)

    def send_email_dialog(self, report_path):
        import tkinter.simpledialog as sd
        sender = sd.askstring("寄件人 Gmail", "輸入 Gmail：")
        password = sd.askstring("應用程式密碼", "輸入 Gmail App Password：", show="*")
        receiver = sd.askstring("收件人", "輸入收件人信箱（如 committee@gmail.com）：")
        try:
            send_email_report(sender, password, receiver, report_path)
            messagebox.showinfo("寄信成功", f"報告已寄出至 {receiver}")
        except Exception as e:
            messagebox.showerror("寄信失敗", f"錯誤：{e}")

if __name__ == "__main__":
    from tkinter import Tk
    root = Tk()
    app = ReconcileApp(root)
    root.mainloop()


ModuleNotFoundError: No module named 'cv2'